In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
data_path = Path("../data/")
gas_fwd = pd.read_csv(data_path / "gas_fwd.csv", parse_dates=True, index_col="date")
gas_fwd.index.name = None
gas_fwd = gas_fwd.reindex(
    pd.date_range(
        gas_fwd.index.min(), gas_fwd.index.max() + pd.offsets.MonthEnd(1), freq="D"
    ),
    method="ffill",
)

gas_fwd["log_price"] = np.log(gas_fwd["price"])
monthly_mean = gas_fwd["log_price"].groupby(gas_fwd.index.month).mean().squeeze()
gas_fwd["monthly_component"] = gas_fwd.index.month.map(monthly_mean).values
gas_fwd["deseasonalized_log"] = gas_fwd["log_price"] - gas_fwd["monthly_component"]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=gas_fwd.index, y=gas_fwd["price"], name="Price"))
fig.add_trace(go.Scatter(x=gas_fwd.index, y=gas_fwd["log_price"], name="Log-price"))
fig.add_trace(
    go.Scatter(x=gas_fwd.index, y=gas_fwd["deseasonalized_log"], name="Deseasonalized")
)
fig = fig.update_layout(template="plotly_dark")
# fig.show()

### Ornstein–Uhlenbeck process

$$
\begin{align*}
dS_t &= \kappa (\mu(t)-S_t)dt+\sigma dW_t \\[1em]

S_t &\dots \text{gas spot price} \\
\mu(t) &\dots \text{seasonal mean level} \\
\kappa &\dots \text{speed of mean reversion} \\
\sigma &\dots \text{volatility} \\
W_t &\dots \text{Wiener process}
\end{align*}
$$
##### Euler-Maruyama discretisation
$$
\begin{align*}
dX_t &= a(X_t,t)dt+b(X_t,t)dW_t \\
0 &= \tau_0 < \tau_1 <\dots<\tau_N = T \\
\Delta t &= T/N \\
Y_0 &= X_0 \\
Y_{n+1} &= Y_n+a(Y_n,\tau_n)\Delta t+b(Y_n,\tau_n) \Delta W_n\\[1em]
\Delta W_n &= W_{\tau_{n+1}} - W_{\tau_n} = \sqrt{\tau_{n+1}-\tau_n}\cdot Z_n = \sqrt{\Delta t} Z_n \\
Z_n &\sim  \mathcal{N}(0,1) \\
\end{align*}
$$

##### Discrete Ornstein–Uhlenbeck process
$$
S_{t+1} = S_t+\kappa (\mu_t-S_t)\Delta t+\sigma \sqrt{\Delta t}Z_t \\[0.5em]
$$

##### Derivation of the drift $\mu$
$$
\begin{align*}
\mathbb{E}[S_t] &= p[t] \dots\text{calibration for actual price} \\
\mathbb{E}[Z_t] &= 0 \\
\mu_t &\dots \text{deterministic} \\[1em]

\mathbb{E}[S_{t+1}] &= \mathbb{E}[S_t] + \kappa(\mu_t-\mathbb{E}[S_t])\Delta t \\
&= (1-\kappa\Delta t) \mathbb{E}[S_t]+\kappa\Delta t \mu_t \\
p_{t+1} &= (1-\kappa\Delta t)p_t + \kappa\Delta t \mu_t \\[1em]

\mu_t &= \dfrac{p_{t+1}-(1-\kappa\Delta t)p_t}{\kappa\Delta t}

\end{align*}
$$

In [ ]:
p_series = gas_fwd["price"]
p = p_series.values

n_steps = len(p_series)
mu = np.zeros(n_steps)
dt = 1.0 / 365.0
kappa = 3.0
sigma = 6.0

for t in range(n_steps - 1):
    mu[t] = (p[t + 1] - (1 - kappa * dt) * p[t]) / (kappa * dt)

# fix last price
mu[-1] = p[-1]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=p_series.index, y=mu, name="Price"))
fig.add_trace(go.Scatter(x=p_series.index, y=p, name="Drift"))
fig = fig.update_layout(
    template="plotly_dark", title="Drift term of the Ornstein-Uhlenbeck process"
)
# fig.show()

In [ ]:
def simulate_gas_spot(
    p: np.ndarray,
    mu: np.ndarray,
    kappa: float,
    sigma: float,
    dt: float,
    n_paths: int = 1000,
) -> np.ndarray:
    n = len(p)
    paths = np.zeros((n_paths, n))
    paths[:, 0] = p[0]

    np.random.seed(42)
    z_dist = np.random.normal(size=(n_paths, n - 1))

    for t in range(n - 1):
        paths[:, t + 1] = (
            paths[:, t]
            + kappa * (mu[t] - paths[:, t]) * dt
            + sigma * np.sqrt(dt) * z_dist[:, t]
        )

    return paths

In [ ]:
paths = simulate_gas_spot(p=p, mu=mu, kappa=kappa, sigma=sigma, dt=dt, n_paths=1000)

sim_mean = paths.mean(axis=0)
gas_fwd["sim_mean"] = sim_mean

In [ ]:
fig = go.Figure()
for path in paths[:1, :]:
    fig.add_trace(
        go.Scatter(
            x=gas_fwd.index, y=path, showlegend=False, line=dict(color="lightgray")
        )
    )
fig.add_trace(
    go.Scatter(
        x=gas_fwd.index, y=gas_fwd["price"], showlegend=False, line=dict(color="red")
    )
)
fig = fig.update_layout(template="plotly_dark", title="Gas spot price paths")
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=gas_fwd.index, y=gas_fwd["price"], name="Price"))
fig.add_trace(go.Scatter(x=gas_fwd.index, y=gas_fwd["sim_mean"], name="Paths mean"))
fig = fig.update_layout(
    template="plotly_dark", title="Mean reversion to the spot price"
)
fig.show()

#### Two-factor model (short + long term):
$$
\begin{align*}
dX_t &= -\kappa X_tdt+\sigma_X dW^X_t \\
dY_t &= \sigma_Y dW^Y_t \\
S_t &= X_t+Y_t+\text{seasonality}
\end{align*}
$$

#### Backward induction:
$$
\mathbb{E}[V_{t+1} \vert S_t,I_t]
$$